In [1]:
!pip install ultralytics

In [2]:
pip install onnxruntime opencv-python

   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB 640.0 kB/s eta 0:00:18
   ---------------------------------------- 0.1/11.3 MB 812.7 kB/s eta 0:00:14
    --------------------------------------- 0.1/11.3 MB 1.1 MB/s eta 0:00:11
    --------------------------------------- 0.3/11.3 MB 1.5 MB/s eta 0:00:08
   - -------------------------------------- 0.5/11.3 MB 2.0 MB/s eta 0:00:06
   -- ------------------------------------- 0.7/11.3 MB 2.6 MB/s eta 0:00:05
   --- ------------------------------------ 0.9/11.3 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.3 MB 3.0 MB/s eta 0:00:04
   ---- ----------------------------------- 1.2/11.3 MB 3.0 MB/s eta 0:00:04
   ---- ----------------------------------- 1.4/11.3 MB 3.2 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/11.3 MB 3.3 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.3 MB 3.4 MB/s eta 0:00:03
  

In [4]:
import onnxruntime as ort

# Load the ONNX model
model_path = "C:/Users/Tech Line/Downloads/ppe/best.onnx"
session = ort.InferenceSession(model_path)

# Print input and output details
print("Input details:")
for input in session.get_inputs():
    print(f"Name: {input.name}, Shape: {input.shape}, Type: {input.type}")

print("\nOutput details:")
for output in session.get_outputs():
    print(f"Name: {output.name}, Shape: {output.shape}, Type: {output.type}")

Input details:
Name: images, Shape: [1, 3, 640, 640], Type: tensor(float)

Output details:
Name: output0, Shape: [1, 9, 8400], Type: tensor(float)


In [9]:
import cv2
import numpy as np
import onnxruntime as ort

# Load the ONNX model
model_path = "C:/Users/Tech Line/Downloads/ppe/best.onnx"
session = ort.InferenceSession(model_path)

# Load an image
image_path = "C:/Users/Tech Line/Downloads/ppe/construction_worker_laborer.jpg"  # Replace with your image path
image = cv2.imread(image_path)

# Preprocess the image
input_shape = session.get_inputs()[0].shape
resized_image = cv2.resize(image, (input_shape[3], input_shape[2]))
input_data = resized_image.transpose(2, 0, 1)
input_data = np.expand_dims(input_data, axis=0).astype(np.float32) / 255.0

# Run inference
outputs = session.run(None, {session.get_inputs()[0].name: input_data})

# Process outputs
output = outputs[0][0]  # Shape: [25200, 85]

for detection in output:
    confidence = detection[4]  # Confidence score
    if confidence > 0.5:  # Confidence threshold
        x_center, y_center, width, height = detection[:4]
        class_scores = detection[5:]
        class_id = np.argmax(class_scores)

        # Convert from normalized coordinates to pixel coordinates
        x1 = int((x_center - width / 2) * image.shape[1])
        y1 = int((y_center - height / 2) * image.shape[0])
        x2 = int((x_center + width / 2) * image.shape[1])
        y2 = int((y_center + height / 2) * image.shape[0])

        # Draw bounding box and label
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"Class {class_id} {confidence:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the image
cv2.imshow("Detection Results", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
import cv2
import numpy as np
import onnxruntime as ort

# Load the ONNX model
model_path = "C:/Users/Tech Line/Downloads/ppe/best.onnx"
session = ort.InferenceSession(model_path)

# Load an image
image_path = "C:/Users/Tech Line/Downloads/ppe/construction_worker_laborer.jpg"  # Replace with your image path
image = cv2.imread(image_path)

# Preprocess the image
input_shape = session.get_inputs()[0].shape
resized_image = cv2.resize(image, (input_shape[3], input_shape[2]))  # Resize to model input size
input_data = resized_image.transpose(2, 0, 1)  # HWC to CHW
input_data = np.expand_dims(input_data, axis=0).astype(np.float32) / 255.0  # Normalize

# Run inference
outputs = session.run(None, {session.get_inputs()[0].name: input_data})

# Process outputs
output = outputs[0][0]  # Shape: [25200, 85]

for detection in output:
    confidence = detection[4]  # Confidence score
    if confidence > 0.3:  # Lower confidence threshold for testing
        x_center, y_center, width, height = detection[:4]
        class_scores = detection[5:]
        class_id = np.argmax(class_scores)

        # Convert from normalized coordinates to pixel coordinates
        x1 = int((x_center - width / 2) * image.shape[1])
        y1 = int((y_center - height / 2) * image.shape[0])
        x2 = int((x_center + width / 2) * image.shape[1])
        y2 = int((y_center + height / 2) * image.shape[0])

        # Draw bounding box and label
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"Class {class_id} {confidence:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the image
cv2.imshow("Detection Results", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
import cv2
import numpy as np
import onnxruntime as ort

# Load the ONNX model
model_path = "C:/Users/Tech Line/Downloads/ppe/best.onnx"
session = ort.InferenceSession(model_path)

# Get input details
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape  # Example: [1, 3, 640, 640]

# Initialize webcam
cap = cv2.VideoCapture(0)  # Use 0 for default webcam

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Preprocess the frame
    resized_frame = cv2.resize(frame, (input_shape[3], input_shape[2]))  # Resize to model input size
    input_data = resized_frame.transpose(2, 0, 1)  # HWC to CHW
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32) / 255.0  # Normalize

    # Run inference
    outputs = session.run(None, {input_name: input_data})

    # Process outputs
    output = outputs[0][0]  # Shape: [25200, 85]

    for detection in output:
        confidence = detection[4]  # Confidence score
        if confidence > 0.5:  # Confidence threshold
            x_center, y_center, width, height = detection[:4]
            class_scores = detection[5:]
            class_id = np.argmax(class_scores)

            # Convert from normalized coordinates to pixel coordinates
            x1 = int((x_center - width / 2) * frame.shape[1])  # Scale to original image width
            y1 = int((y_center - height / 2) * frame.shape[0])  # Scale to original image height
            x2 = int((x_center + width / 2) * frame.shape[1])
            y2 = int((y_center + height / 2) * frame.shape[0])

            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Class {class_id} {confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Real-time Detection", frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [13]:
import cv2
import numpy as np
import onnxruntime as ort

# Load the ONNX model
model_path = "C:/Users/Tech Line/Downloads/ppe/best.onnx"
session = ort.InferenceSession(model_path)

# Print model input/output details
print("Input details:")
for inp in session.get_inputs():
    print(f"Name: {inp.name}, Shape: {inp.shape}, Type: {inp.type}")

print("\nOutput details:")
for out in session.get_outputs():
    print(f"Name: {out.name}, Shape: {out.shape}, Type: {out.type}")

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Debug: Show original frame dimensions
    print(f"Original frame shape: {frame.shape}")

    # Resize to model input dimensions
    input_name = session.get_inputs()[0].name
    input_shape = session.get_inputs()[0].shape
    
    try:
        # Resize frame
        resized_frame = cv2.resize(frame, (input_shape[3], input_shape[2]))
        
        # Normalize and prepare input
        input_data = resized_frame.astype(np.float32) / 255.0
        input_data = input_data.transpose(2, 0, 1)  # HWC to CHW
        input_data = np.expand_dims(input_data, axis=0)

        # Print input data details
        print(f"Prepared input shape: {input_data.shape}")

        # Run inference
        outputs = session.run(None, {input_name: input_data})

    except Exception as e:
        print(f"Error during inference: {e}")
        break

    # Display frame
    cv2.imshow("Frame", frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Input details:
Name: images, Shape: [1, 3, 640, 640], Type: tensor(float)

Output details:
Name: output0, Shape: [1, 9, 8400], Type: tensor(float)
Original frame shape: (480, 640, 3)
Prepared input shape: (1, 3, 640, 640)
Original frame shape: (480, 640, 3)
Prepared input shape: (1, 3, 640, 640)
Original frame shape: (480, 640, 3)
Prepared input shape: (1, 3, 640, 640)
Original frame shape: (480, 640, 3)
Prepared input shape: (1, 3, 640, 640)
Original frame shape: (480, 640, 3)
Prepared input shape: (1, 3, 640, 640)
Original frame shape: (480, 640, 3)
Prepared input shape: (1, 3, 640, 640)
Original frame shape: (480, 640, 3)
Prepared input shape: (1, 3, 640, 640)
Original frame shape: (480, 640, 3)
Prepared input shape: (1, 3, 640, 640)
Original frame shape: (480, 640, 3)
Prepared input shape: (1, 3, 640, 640)
Original frame shape: (480, 640, 3)
Prepared input shape: (1, 3, 640, 640)
Original frame shape: (480, 640, 3)
Prepared input shape: (1, 3, 640, 640)
Original frame shape: (480, 